In [ ]:
import ibis
from ibis import _

In [ ]:
t = ibis.read_parquet("./cleaned_receipts.parquet")
print(t.count())
t

In [ ]:
# order from changing-least-quickly to changing-most-quickly
# so that we are most likely to be able to get long runs of the same value
# to compress well
t = t.select(
    state=_.from__state.nullif(""),
    city=_.from__city.nullif(""),
    postcode=_.from__zipcode.nullif(""),
    street=_.from__street1.nullif(""),
    unit=_.from__street2.nullif(""),
)
t = t.filter(~_.street.upper().contains("PO BOX"))
t = t.order_by(t.columns)
t = t.mutate(record_id=ibis.row_number()).relocate("record_id")
t

In [ ]:
t.to_parquet("alaska-addresses-dirty.parquet", compression="zstd", compression_level=15)